In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/'
!ls

/content/gdrive/.shortcut-targets-by-id/1-WHOINXFZ4JSjL7g5g092rZ8rCZfdBPw/odir-ivashnyov/ODIR
CrossValidationScore.py		  Predictions.py
cv_split.pickle			  predict.py
data				  __pycache__
EDA.ipynb			  README.md
initial_test_b3_fold3_clahe	  split_folds.ipynb
logs				  submit
logs_final			  submit_final
MNIST				  submit_initial_test_b3.csv
model_concate_10_no_epoch.txt	  submit_oanh_initial_test_b3.csv
model_concate_15epoch_weight.txt  submit_vessel_initial_test_b3.csv
model_concate_19epoch.txt	  train_1cycle.py
model_concate_20epoch.txt	  train_all_folds_args.sh
model_concate_5_no_epoch.txt	  train_all_folds.sh
model_concate_6epoch.txt	  train.py
model_concate_epoch_20.txt	  utils.py
odir_submit.py			  Visualization.ipynb


In [ ]:
!pip install catalyst==20.2.4
!pip install tqdm==4.33
!pip install efficientnet_pytorch
!pip install pytorch_toolbelt

In [ ]:
from sklearn import svm
from keras.callbacks import CSVLogger
from keras.callbacks import LambdaCallback
from utils import *
#from simple_model import *
#from preprocess import *
#from make_labels import *
import pandas as pd
import sys
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf
import torch
import collections
from pytorch_toolbelt.inference import tta
from catalyst.dl.callbacks import InferCallback
from catalyst.dl.runner import SupervisedRunner
from torch.nn.functional import softmax
from catalyst.dl.callbacks import EarlyStoppingCallback, AccuracyCallback, F1ScoreCallback, ConfusionMatrixCallback, MixupCallback
from catalyst.contrib.nn.schedulers import OneCycleLR, ReduceLROnPlateau, StepLR, MultiStepLR
from torchvision import transforms
import torch.nn.functional as F
from torch.autograd import Variable
from torchsummary import summary
import pickle
import time
import copy


Using TensorFlow backend.
alchemy not available, to install alchemy, run `pip install alchemy-catalyst`.


In [ ]:
data = pd.read_csv(
    '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/splited_train_new.csv')
splits = pickle.load(open(
    '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/cv_split.pickle', 'rb'))
labels = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
image_size = 240
fold_idx = 3
batch_size = 1
lr = 1e-5

In [ ]:
train_path1 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/clahe-train/'
valid_path1 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/clahe-train/'

train_dataset1 = EyeDataset(dataset_path=train_path1,
                           labels=data.loc[splits['train_idx'][fold_idx],labels].values,
                           ids=data.loc[splits['train_idx'][fold_idx],labels].values,
                           albumentations_tr=aug_train_heavy(image_size))

train_loader1 = DataLoader(train_dataset1,
                          num_workers=8,
                          pin_memory=False,
                          batch_size=batch_size,
                          shuffle=False)
logdir = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/initial_test_b3_fold3_clahe/'
modelA = prepare_model('efficientnet-b3', 8)
modelA.cuda()
modelA.load_state_dict(torch.load(os.path.join(
    logdir, '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/initial_test_b3_fold3_clahe/checkpoints/best.pth'))['model_state_dict'])
modelA.eval()

In [ ]:
train_path2 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/vessel-train-full/'
valid_path2 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/vessel-train-full/'

train_dataset2 = EyeDataset(dataset_path=train_path2,
                           labels=data.loc[splits['train_idx'][fold_idx],labels].values,
                           ids=data.loc[splits['train_idx'][fold_idx],labels].values,
                           albumentations_tr=aug_train_heavy(image_size))
train_loader2 = DataLoader(train_dataset2,
                          num_workers=8,
                          pin_memory=False,
                          batch_size=batch_size,
                          shuffle=False)

logdir = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/initial_test_b3_fold3_vessel/'
modelB = prepare_model('efficientnet-b3', 8)
modelB.cuda()
modelB.load_state_dict(torch.load(os.path.join(
    logdir, '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/initial_test_b3_fold3_vessel/checkpoints/best.pth'))['model_state_dict'])
modelB.eval()

In [ ]:
for i2 in train_loader2:
  input2,target2 = i2
  #print(input1)
  #print(target1)
  print(i2)
  input2 = input2.cuda()
  break

TypeError: ignored

In [ ]:
import sklearn
class NeuralNet(nn.Module):
    def __init__(self, modelA, modelB, nb_classes=8):
        super(NeuralNet, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.drop = nn.Dropout(0.3)
        self.out2 = nn.Linear(1536, 8)
        self.modelA.fc = nn.Identity()
        self.modelB.fc = nn.Identity()

    def forward(self, x1, x2):
        x1 = self.modelA.extract_features(x1)
        x2 = self.modelB.extract_features(x2)
        
        x1 = nn.AdaptiveAvgPool2d((1, 1))(x1)
        x2 = nn.AdaptiveAvgPool2d((1, 1))(x2)
        x1 = x1.view(1,1536)
        x1 = torch.Tensor.cpu(x1).detach().numpy()
        x2 = x2.view(1,1536)
        x2 = torch.Tensor.cpu(x2).detach().numpy()
        x = x1+x2        
        sigmoid = tf.math.sigmoid(x)
        multi = tf.math.multiply(sigmoid,x2)
        add = multi + x1
        add = add.numpy()
        x = torch.from_numpy(add)
        x = self.drop(x)
        x = x.flatten()
        x = self.out2(x)
        return x

In [ ]:
probabilities = []
x_model = NeuralNet(modelA, modelB)
y = x_model(input1,input2)
print(y)
print(type(y))
y = torch.Tensor.cpu(y).detach().numpy()
print(y)
print(type(y))
probabilities = softmax(torch.from_numpy(y),dim=0).numpy()
print(probabilities)

In [ ]:
optimizer = torch.optim.Adam(x_model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer=optimizer, factor=0.75, patience=2)
loss_log = []
PATH_SAVE = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/clahe_vessel_forgetgate/'
if not os.path.exists(PATH_SAVE):
       os.makedirs(PATH_SAVE)
for epoch in range(20):
  print('====Epoch ',epoch,": ")
  for i1, i2 in zip(enumerate(train_loader1),enumerate(train_loader2)):
    #print(i1)
    stt1, data1 = i1
    input1,target1 = data1
    #print(i2)
    stt2, data2 = i2
    input2,target2 = data2
    # zero the parameter gradients
    optimizer.zero_grad()
    input1 = input1.cuda()
    input2 = input2.cuda()

    # forward + backward + optimize
    outputs = x_model(input1,input2)
    #print(outputs)
    #print(target1)
    outputs = outputs.view(1,8)

    loss = criterion(outputs, target1)
    loss.backward()
    #print(loss)
    optimizer.step()
  loss_log.append(loss)
  if (epoch>=15):
    torch.save(x_model.state_dict(), PATH_SAVE + 'clahe_vessel_forgetgate_epoch_'+str(epoch)+'.pth')

with open(PATH_SAVE+"clahe_vessel_forgetgate.txt", "w") as output:
    output.write(str(loss_log))

====Epoch  0 : 
tensor(5.2391, grad_fn=<NllLossBackward>)
====Epoch  1 : 
tensor(7.0463, grad_fn=<NllLossBackward>)
====Epoch  2 : 
tensor(6.8624, grad_fn=<NllLossBackward>)
====Epoch  3 : 
tensor(6.9787, grad_fn=<NllLossBackward>)
====Epoch  4 : 
tensor(6.8653, grad_fn=<NllLossBackward>)
====Epoch  5 : 
tensor(4.8871, grad_fn=<NllLossBackward>)
====Epoch  6 : 
tensor(5.2329, grad_fn=<NllLossBackward>)
====Epoch  7 : 
tensor(3.8114, grad_fn=<NllLossBackward>)
====Epoch  8 : 
tensor(8.5890, grad_fn=<NllLossBackward>)
====Epoch  9 : 
tensor(4.5288, grad_fn=<NllLossBackward>)
====Epoch  10 : 
tensor(5.3942, grad_fn=<NllLossBackward>)
====Epoch  11 : 
tensor(5.5714, grad_fn=<NllLossBackward>)
====Epoch  12 : 
tensor(8.2257, grad_fn=<NllLossBackward>)
====Epoch  13 : 
tensor(3.5530, grad_fn=<NllLossBackward>)
====Epoch  14 : 
tensor(6.2300, grad_fn=<NllLossBackward>)


In [ ]:
x_model.load_state_dict(torch.load(os.path.join(
        '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/clahe_vessel_forgetgate/clahe_vessel_forgetgate_epoch_19.pth')))
x_model.eval()

test_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/XYZ_ODIR.csv')
test_data_left = test_data.copy()
test_data_right = test_data.copy()
test_data_left.loc[:, 'id'] = test_data_left.ID.apply(
    lambda x: str(x)+'_left.jpg')
test_data_right.loc[:, 'id'] = test_data_left.ID.apply(
    lambda x: str(x)+'_right.jpg')
test_data = pd.concat([test_data_left, test_data_right])
test_data.sort_values(['ID'], inplace=True)
test_path = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/clahe-test/'
test_dataset = EyeDataset(dataset_path=test_path,
                          labels=test_data.loc[:, labels].values,
                          ids=test_data.loc[:, 'id'].values,
                          albumentations_tr=aug_val(image_size))
test_loader = DataLoader(test_dataset,
                         num_workers=8,
                         pin_memory=False,
                         batch_size=batch_size,
                         shuffle=False)


test_path1 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/vessel-test-full/'
test_dataset_vessel = EyeDataset(dataset_path=test_path1,
                                 labels=test_data.loc[:, labels].values,
                                 ids=test_data.loc[:, 'id'].values,
                                 albumentations_tr=aug_val(image_size))
test_loader_vessel = DataLoader(test_dataset_vessel,
                                num_workers=8,
                                pin_memory=False,
                                batch_size=batch_size,
                                shuffle=False)

probabilities = []
probabilities_list = []
exp_name = 'initial_test_b3'
optimizer = torch.optim.Adam(x_model.parameters(), lr=1e-5)

for i1, i2 in zip(enumerate(test_loader), enumerate(test_loader_vessel)):
    # print(i1)
    stt1, data1 = i1
    input1, target1 = data1
    # print(i2)
    stt2, data2 = i2
    input2, target2 = data2
    # zero the parameter gradients
    optimizer.zero_grad()
    input1 = input1.cuda()
    input2 = input2.cuda()

    # forward + backward + optimize
    outputs = x_model(input1, input2)
    outputs = torch.Tensor.cpu(outputs).detach().numpy()
    probabilities = softmax(torch.from_numpy(outputs), dim=0).numpy()
    # print(probabilities)
    for idx in range(probabilities.shape[0]):
        if all(probabilities[:] < 0.5):
            probabilities[0] = 1.0
    probabilities_list.append(probabilities)
arr_list = []
arr = np.array(probabilities_list)
arr_list.append(arr)
print(arr_list)


probabilities_combined = np.stack(arr_list, axis=0).mean(axis=0)
predicted_labels = pd.DataFrame(probabilities_combined, columns=labels)
predicted_labels['id'] = test_data.loc[:, 'id'].values
predicted_labels.loc[:, 'ID'] = predicted_labels.id.apply(
    lambda x: x.split('_')[0])
predicted_labels_groupped = predicted_labels.groupby(
    ['ID']).aggregate(dict(zip(labels, ['max']*(len(labels)))))
print(type(predicted_labels_groupped))
predicted_labels_groupped['ID'] = predicted_labels_groupped.index.values.astype(
    int)
predicted_labels_groupped.reset_index(drop=True, inplace=True)
predicted_labels_groupped.sort_values('ID', inplace=True)
predicted_labels_groupped = predicted_labels_groupped.loc[:, ['ID']+labels]
predicted_labels_groupped.to_csv(
    '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/submit_final/clahe_vessel_forgetgate_epoch_19.csv', index=False)

NameError: ignored